In [ ]:
# import google drive
from google.colab import drive

# mount google drive
drive.mount('/content/drive')


In [ ]:
# unzip data.zip
!unzip "/content/drive/MyDrive/studies/ai-project/data.zip" -d /content/data_dir


In [ ]:
# import
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import matplotlib.pyplot as plt
import random
import cv2
from sklearn.model_selection import train_test_split


In [ ]:
# data_dir
data_dir = "/content/data_dir"
data_dir = pathlib.Path(data_dir)


In [ ]:
# image_count
image_count = len(list(data_dir.glob('*/*.*')))


In [ ]:
# configs
batch_size = 3
img_height = 128
img_width = 128


In [ ]:
# class names
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))


In [ ]:
# imagePaths
imagePaths = []

for k, class_name in enumerate(class_names):
    for f in os.listdir(str(data_dir)+"/"+class_name):
        imagePaths.append([str(data_dir)+"/"+class_name+'/'+f, k])

random.shuffle(imagePaths)


In [ ]:
# data and labels
data = []
labels = []

for imagePath in imagePaths:
    image = cv2.imread(imagePath[0])
    image = cv2.resize(image, (img_width, img_height))
    data.append(image)
    
    label = imagePath[1]
    labels.append(label)


In [ ]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)


In [ ]:
# debug
plt.subplots(3,4)
for i in range(12):
    plt.subplot(3,4, i+1)
    plt.imshow(data[i])
    plt.axis('off')
    plt.title(class_names[labels[i]])
plt.show()


In [ ]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
# create model
model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),

  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(1024, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(1024, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(512, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(256, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid'),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(len(class_names))
])


In [ ]:
# compile model
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [ ]:
# train model
epochs = 3
model.fit(
  trainX,
  trainY,
  epochs=epochs
)


In [ ]:
from numpy import argmax
from sklearn.metrics import confusion_matrix, accuracy_score

pred = model.predict(testX)
predictions = argmax(pred, axis=1) # return to label

cm = confusion_matrix(testY, predictions)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Model confusion matrix')
fig.colorbar(cax)
ax.set_xticklabels(['', *class_names])
ax.set_yticklabels(['', *class_names])

for i in range(len(class_names)):
    for j in range(len(class_names)):
        ax.text(i, j, cm[j, i], va='center', ha='center')

plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


accuracy = accuracy_score(testY, predictions)
print("Accuracy : %.2f%%" % (accuracy*100.0))

In [ ]:
# # train dataset
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   data_dir,
#   validation_split=0.2,
#   subset="training",
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)
  

In [ ]:
# # validation dataset
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   data_dir,
#   validation_split=0.2,
#   subset="validation",
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)
  

In [ ]:
# # class names
# class_names = train_ds.class_names


In [ ]:
# # debug
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(3):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")


In [ ]:
# # debug
# for image_batch, labels_batch in train_ds:
#   print(image_batch.shape)
#   print(labels_batch.shape)
#   break


In [ ]:
# # serialize data
# normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
# normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))


In [ ]:
# # debug
# first_image = image_batch[0]
# print(np.min(first_image), np.max(first_image))


In [ ]:
# # debug
# plt.figure(figsize=(10, 10))
# for i in range(3):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].numpy().astype("uint8"))
#   plt.title(class_names[labels_batch[i]])
#   plt.axis("off")


In [ ]:
# # performance
# AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
# # create model
# model = tf.keras.Sequential([
#   tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
#   tf.keras.layers.Conv2D(32, 3, activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Conv2D(32, 3, activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Conv2D(32, 3, activation='relu'),
#   tf.keras.layers.MaxPooling2D(),
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(len(class_names))
# ])


In [ ]:
# # compile model
# model.compile(
#   optimizer='adam',
#   loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
#   metrics=['accuracy'])


In [ ]:
# # train model
# epochs = 3
# model.fit(
#   train_ds,
#   validation_data=val_ds,
#   epochs=epochs
# )
